# Building Register Circuits 

- `RegisterBoxes` and `RegisterCircuits` are the central feature of qtnmtts
- They allow abstract dvelopment of quantum algorithms, just working with quantum registers and boxes
- This is expands on pytket's `Circuit` class, which just works with canonical qubits when doing `CircBox` composition


In [1]:
from qtnmtts.circuits.core import RegisterCircuit
from qtnmtts.circuits.core import RegisterBox
from pytket.circuit import CircBox, Circuit, QubitRegister
from dataclasses import dataclass

## Qubit Register

- `QubitRegisters` are essential to qtnmttss, they can be initialised by:

In [2]:
QubitRegister('q', 2)

QubitRegister("q", 2)

## RegisterCircuits

- `RegisterCircuits` are a subclass of `Circuit`, and can be used in the same way and inhertit all the same methods
- It has an extended `add_registerbox()` method, which allows you to add an oraclebox to the registers of the `RegisterCircuit`
- They are also compatible with pytket circuits

In [3]:
register_circuit = RegisterCircuit()

n_a_qubits = 2
n_b_qubits = 2

x_qreg = register_circuit.add_q_register('x', n_a_qubits)
y_qreg = register_circuit.add_q_register('y', n_b_qubits)

## RegisterBox

- `RegisterBoxes` keep track of the `QubitRegisters` when adding boxes, something pytket cannot do because every circbox must be created with a flattened circuit
- This means that you can add a `RegisterBox` to a `RegisterCircuit`, and it will automatically add the registers to the correct ordering of qubits inside the `RegisterBox` to the `RegisterCircuit` with the `QRegMap` Class


### QRegs
- Each `RegisterBox` must have a corresponding `QRegs` dataclass
- This has attributes which are `QubitRegisters` inside the `RegisterBox`
- The attributes are given meaningful names in context with the register box. e.g. `ancilla`, `control` or `state`
- These attribute QubitRegisters have their own `name` attribute which is the name of the pytket register. Eg the default for the `state` register is typically `q`
- This is different to the pytket Circuit `.q_registers` which is a list of all the registers in the circuit. Whereas the `QRegs` dataclass allows one to directly access the registers inside the `RegisterBox` by attribute name
- The `QRegs` dataclass is a property of the base `RegisterBox` class, and is automatically created when the `RegisterBox` is constructed, hence all registers can be accessed by attribute name such as `box.qregs.state` or `box.qregs.ancilla` etc
- Lists of qubit registers are also supported as dataclass attributes.

### Construction
- `RegisterBoxes` can be constructed in the same way as `CircBoxes` but with the addition of a `QReg` class
- If `QubitRegisters` are created inside the `RegisterBox` they must be defined as attributes of the `QRegs` dataclass
- The `QRegs` class must be passed into the `super().__init__()` method
- The `RegisterCircuit` must be passed into the `super().__init__()` method
- If the `RegisterBox` is constructed with any sub boxes, they should be passed in as variables into the constructor and the should be set as properties
- This allows the `RegisterBox` to be built with composition and is also future proof for new compilation strategies and boxes.
- `postselect` is either empty or a dict of qubits to postselect with the measurement outcome, defined in the constructor

In [4]:
@dataclass
class ExampleQRegs:
    """QRegister Names dataclass for ExampleRegisterBox

    Each register in the box has a descriptor given by the attribute name
    The attribute value is the name of the register
    """
    a: QubitRegister
    b: QubitRegister

class ExampleRegisterBox(RegisterBox):
    
    def __init__(self,sub_box_a:CircBox, sub_box_b:CircBox, qreg_a:QubitRegister, qreg_b:QubitRegister):

        self._sub_box_a = sub_box_a
        self._sub_box_b = sub_box_b

        circ = RegisterCircuit(self.__class__.__name__)  
        circ.add_q_register(qreg_a) 
        circ.add_q_register(qreg_b)

        qregs = ExampleQRegs(qreg_a, qreg_b)

        circ.add_gate(self.sub_box_a, qreg_a)
        circ.add_gate(self.sub_box_b, qreg_b)

        self._postselect = {qreg_a: 0}

        super().__init__(qregs, circ)

    @property
    def sub_box_a(self):
        """sub_box_a to be defined in child class"""
        return self._sub_box_a

    @property
    def sub_box_b(self):
        """sub_box_b to be defined in child class"""
        return self._sub_box_b
    
    @property
    def postselect(self):
        """postselect to be defined in child class"""
        return self._postselect 


## Register Box Composition
- The RegisterBoxes is now constructed with composition of two pytket.Circbox in `ExampleBox`
- This can also be done with more complicated `RegisterBoxes`
- Look at the LCUBoxes for advanced usage

In [18]:
from pytket.circuit.display import render_circuit_jupyter

a_qregs = QubitRegister('c', 2) # Can be any str
b_qregs = QubitRegister('d', 2)

box0 = CircBox(Circuit(a_qregs.size, 'CustomA'))
box1 = CircBox(Circuit(b_qregs.size, 'CustomB'))

custom_box = ExampleRegisterBox(box0, box1, a_qregs, b_qregs)
render_circuit_jupyter(custom_box.get_circuit())

- There should only be 2 levels of inheritance in the box hierarchy. The first level RegisterBox subclass where the circuit composition is done, the second is the concrete subclass.

### Register Mapping

- The `QRegMap` class is used to map the `QubitRegisters` inside the `RegisterBox` to the `QubitRegisters` inside the global `RegisterCircuit`
- This handles correct qubit ordering and size of qubit registers
- Errors are raised if the `QubitRegisters` are not compatible

- `QRegMap` maps one list of `QubitRegisters` to another list of `QubitRegisters` in a one to one mapping

- `QRegMap.from_dict()` is a class method which takes a more explicit dictionary of {box.qreg: oracle_circuit.qreg} and returns a `QRegMap` object

In [19]:
from qtnmtts.circuits.core import QRegMap
qreg_map = QRegMap(custom_box.q_registers, register_circuit.q_registers)
qreg_map

QRegMap (box -> circ):

QREG: c [2] ->                     x [2]
QREG: d [2] ->                     y [2]

In [20]:
QRegMap.from_dict({custom_box.qreg.a: x_qreg, custom_box.qreg.b :y_qreg})

QRegMap (box -> circ):

QREG: c [2] ->                     x [2]
QREG: d [2] ->                     y [2]

In [21]:
register_circuit_copy = register_circuit.copy()

register_circuit_copy.add_registerbox(custom_box, qreg_map)
render_circuit_jupyter(register_circuit_copy)

### Initialise Circuit
- If we just want a register of circuits, which is the same as the ones in the RegisterBox we can use the `initialise_circuit()` method

In [22]:
init_circ = custom_box.initialise_circuit()
render_circuit_jupyter(init_circ)

A QRegMap is not needed:
- If the `RegisterBox` has the same registers as the `RegisterCircuit` with the same number of qubits

A box can be added to a subset of registers in this way


In [23]:
init_circ = custom_box.initialise_circuit()
init_circ.add_registerbox(custom_box)
render_circuit_jupyter(init_circ)

## PowerBoxes

- We can generate a power of a box with the `power()` method
- This corresponds to the box being repeated n times
- This will clone all the register properties of the input box
- This can be over written by the child classes if a better method is known

In [24]:
power_box = custom_box.power(2)
print(power_box.register_box.qreg.a)
render_circuit_jupyter(power_box.reg_circuit)

c


## QControlBoxes

- This will generate a box which is controlled on an ancilla register
- It defaults to the pytket QControlBox, but can be overwritten by the child class if a better method is known

In [25]:
qcontorlbox =  custom_box.qcontrol(1)
render_circuit_jupyter(qcontorlbox.get_circuit())

## Post-Selection

- If a box has post selection, which can used on measurement this can be done with the `post_select()` method

In [26]:
custom_box.postselect

{QubitRegister("c", 2): 0}

## From Pytket Circuit

- We can initialise a register box from a pytket circuit
- Where the default behaviour is that the qreg dataclass attributes are the pytket qubit register names

In [27]:
circuit = Circuit()
qreg1 = circuit.add_q_register("a", 2)
qreg2 = circuit.add_q_register("b", 3)
circuit.H(qreg1[0])
circuit.CX(qreg1[0], qreg2[1])

# Call the from_Circuit method
register_box = RegisterBox.from_Circuit(circuit)
register_box.qreg

QRegs(a=QubitRegister("a", 2), b=QubitRegister("b", 3))

- user defined data class attributes can be support by supplying a dict[str, QubitRegister] to the `qregs_attrs` argument

In [28]:
qregs_attrs = {'x': qreg1, 'y': qreg2}
register_box = RegisterBox.from_Circuit(circuit, qregs_attrs)
register_box.qreg

QRegs(x=QubitRegister("a", 2), y=QubitRegister("b", 3))

## From Pytket `CircBox`

- As well as being able to initialise `RegisterBoxes` from `RegisterCircuits`, we can also initialise `RegisterBoxes` from pytket `CircBoxes`
- This is advised if you want to use a pytket `CircBox` in a `RegisterCircuit` when the pytket `CircBox` connects to multiple registers. 
- This conversion is important because `CircBoxes` only have a single register.
- Once converted, the `RegisterBox` can be used in a `RegisterCircuit` as normal with `QRegMap` for extra control.

- The default behaviour converts the single register of the pytket `CircBox` to a `QubitRegister` with the name attribute name `default` and the symbol 'q'

In [29]:
circ = CircBox(Circuit(4).Ry(0.5,0).CX(0, 1).CX(1, 2).CX(2, 3).Ry(0.5,3))
reg_box = RegisterBox.from_CircBox(circ)
reg_box.qreg

QRegs(default=QubitRegister("q", 4))

- We can also assign new register names to the circuit in the register box
- We provide the assign register names as a dictionary of {new_name: [qubit_indices]}
- The qubit indices are the indices of the qubits in the single pytket `CircBox` register
- Importantly, the attributes of the `QRegs` dataclass will be same as the keys of the dictionary

In [30]:
assign_registers = {'a':[0,1], 'b':[2,3]}
reg_box = RegisterBox.from_CircBox(circ, assign_registers)
reg_box.qreg

QRegs(a=QubitRegister("a", 2), b=QubitRegister("b", 2))